In [1]:
from functions import *

/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [2]:
psl_all = xr.open_dataset(atm_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.cam.h0.PSL.040101-050012.nc')['PSL']
sw = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.FSA.040101-050012.nc')["FSA"]
lw = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.FIRA.040101-050012.nc')["FIRA"]
sh = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.FSH.040101-050012.nc')["FSH"]
lh = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.QSOIL.040101-050012.nc')["QSOIL"]/(2835*1000*1000*(1/1000)) # m/s / lhro
fgr = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.FGR.040101-050012.nc')["FGR"]
fsm = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.FSM.040101-050012.nc')["FSM"]
snow = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.SNOW.040101-050012.nc')["SNOW"]/1000 #m/s
qice = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.QICE.040101-050012.nc')["QICE"]/1000 #m/s
pct_ice = xr.open_dataset(ice_dir + 'surfdata_fv09_hist_16pfts_nourb_CMIP6_21ka.c200624.nc')["PCT_GLACIER"]
lh = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.QSOIL.040101-050012.nc')["QSOIL"]/(2835*1000*1000*(1/1000)) # m/s / lhro
e = xr.open_dataset(lnd_dir_100 + 'b.e21.B1850CLM50SP.f09_g17.21ka.01.clm2.h0.QSOIL.040101-050012.nc')["QSOIL"]/1000


In [3]:
gh = xr.DataArray(fsm - fgr)
ps = xr.DataArray(snow)
seb = (sw - lw - sh - lh + gh)
melt = xr.where(seb > 0, seb / (334 * 1000 * 1000), 0)
smb = ps - melt - e
ice_mask = np.tile(xr.where(pct_ice == 100, 1, np.nan), (smb.shape[0], 1, 1))
smb_all = smb*ice_mask

In [4]:
print("______________")
print("PROCESSING DATA")
north_atlantic_ocean_psl = preprocess_na_ocean(psl_all)
print("______________")
print("CONVERTING TO 2D")
time_length, lat_length, long_length = get_lengths(north_atlantic_ocean_psl)
X_dimensional = north_atlantic_ocean_psl.values.reshape(time_length, lat_length * long_length)
print("______________")

______________
PROCESSING DATA
converting to hecta pascals
deseasonalizing
slicing along lat=(20,80) and long=(-90,40)
______________
CONVERTING TO 2D
time 1200 lat 64 long 105
______________


In [5]:
data = xr.Dataset(
    {
        "data": (["data", "month"], X_dimensional),
        "lat_length": ([], lat_length),
        "long_length": ([], long_length),
        "original": (["t", "lat", "lon"], north_atlantic_ocean_psl.values),
    },
    coords={
        "t": north_atlantic_ocean_psl.time,
        "lat": north_atlantic_ocean_psl.lat,
        "lon": north_atlantic_ocean_psl.lon,
    }    
)

data.to_netcdf("psl_100_eq.nc")
print("data saved")

data saved


In [6]:
print("______________")
print("PROCESSING DATA")
smb_processed = preprocess_laurentide(smb_all)
print("______________")
print("CONVERTING TO 2D")
smb_X_dimensional, smb_longs, smb_lats, smb_long_length, smb_lat_length = nans_3d_to_2d(smb_processed)
print("______________")

______________
PROCESSING DATA
converting data to m/s
deseasonalizing data
slicing data
______________
CONVERTING TO 2D
time 1200 lat 64 long 101
finished finding nans
total nans 4578
up through 0
up through 50
up through 100
up through 150
up through 200
up through 250
up through 500
up through 550
up through 600
up through 850
up through 900
up through 950
up through 1000
up through 1050
up through 1100
up through 1150
X shape (1886, 1200)
______________


In [7]:
data = xr.Dataset(
    {
        "data": (["data", "month"], smb_X_dimensional),
        "long_pos":  (["long_pos"], smb_longs), 
        "lat_pos":(["lat_pos"], smb_lats),
        "long_length": ([], smb_long_length),
        "lat_length": ([], smb_lat_length),
        "original": (["t", "lat", "lon"], smb_processed.values),
    },
    coords={
        "t": smb_processed.time,
        "lat": smb_processed.lat,
        "lon": smb_processed.lon,
    }   
    
)

data.to_netcdf("laurentide_smb_100_eq.nc")
print("data saved")

data saved


In [8]:
print("______________")
print("PROCESSING DATA")
smb_processed = preprocess_greenland(smb_all)
print("______________")
print("CONVERTING TO 2D")
smb_X_dimensional, smb_longs, smb_lats, smb_long_length, smb_lat_length = nans_3d_to_2d(smb_processed)
print("______________")

______________
PROCESSING DATA
converting data to m/s
deseasonalizing data
slicing data
______________
CONVERTING TO 2D
time 1200 lat 34 long 52
finished finding nans
total nans 1207
up through 0
up through 50
up through 100
up through 150
up through 200
up through 250
up through 300
up through 350
up through 400
up through 450
up through 500
up through 550
up through 600
up through 650
up through 700
up through 750
up through 800
up through 850
up through 900
up through 950
up through 1000
up through 1050
up through 1100
up through 1150
X shape (561, 1200)
______________


In [9]:
data = xr.Dataset(
    {
        "data": (["data", "month"], smb_X_dimensional),
        "long_pos":  (["long_pos"], smb_longs), 
        "lat_pos":(["lat_pos"], smb_lats),
        "long_length": ([], smb_long_length),
        "lat_length": ([], smb_lat_length),
        "original": (["t", "lat", "lon"], smb_processed.values),
    },
    coords={
        "t": smb_processed.time,
        "lat": smb_processed.lat,
        "lon": smb_processed.lon,
    }   
    
)

data.to_netcdf("greenland_smb_100_eq.nc")
print("data saved")

data saved


In [10]:
print("______________")
print("PROCESSING DATA")
smb_processed = preprocess_eurasia(smb_all)
print("______________")
print("CONVERTING TO 2D")
smb_X_dimensional, smb_longs, smb_lats, smb_long_length, smb_lat_length = nans_3d_to_2d(smb_processed)
print("______________")

______________
PROCESSING DATA
converting data to m/s
deseasonalizing data
slicing data
______________
CONVERTING TO 2D
time 1200 lat 43 long 102
finished finding nans
total nans 3405
up through 0
up through 50
up through 100
up through 150
up through 200
up through 250
up through 300
up through 350
up through 400
up through 450
up through 500
up through 550
up through 600
up through 650
up through 700
up through 750
up through 800
up through 850
up through 900
up through 950
up through 1000
up through 1050
up through 1100
up through 1150
X shape (981, 1200)
______________


In [11]:
data = xr.Dataset(
    {
        "data": (["data", "month"], smb_X_dimensional),
        "long_pos":  (["long_pos"], smb_longs), 
        "lat_pos":(["lat_pos"], smb_lats),
        "long_length": ([], smb_long_length),
        "lat_length": ([], smb_lat_length),
        "original": (["t", "lat", "lon"], smb_processed.values),
    },
    coords={
        "t": smb_processed.time,
        "lat": smb_processed.lat,
        "lon": smb_processed.lon,
    }   
    
)

data.to_netcdf("eurasia_smb_100_eq.nc")
print("data saved")

data saved
